# **Classify Generated Samples (Feedback Simulation)**

In [1]:
import torch
import networks as nws
import utils
import os
import matplotlib.pyplot as plt

In [2]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
print(device)

nz = 128
version = 3
G_final = nws.Generator(version=version, nz=nz).to(device)

original_train_epochs = 300
original_save_dir = "checkpoints_original"
G_final.load_state_dict(torch.load(f'{original_save_dir}/gen_MNIST_full_v{version}_nz={nz}_epochs={original_train_epochs}.pth'))

classifier = nws.CNN_Classifier().to(device)
classifier.load_state_dict(torch.load(f'classifier/mnist_classifier.pth'))

cuda:0


C:\Users\Admin\AppData\Local\Temp\ipykernel_6056\2297317714.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  G_final.load_state_dict(torch.load(f'{original_save_dir}/gen

<All keys matched successfully>

In [3]:
generated = utils.generate_folder(G_final, nz, device, examples=5000)
predicted_labels = utils.classify(classifier, generated, device)

In [ ]:
target_class = 8  # `1` or `8`
undesired_folder = f"MNIST_ul{target_class}_data/{target_class}/data"
desired_folder = f"MNIST_ul{target_class}_data/no{target_class}/data"
os.makedirs(undesired_folder, exist_ok=True)
os.makedirs(desired_folder, exist_ok=True)

for i, (data, label) in enumerate(zip(generated, predicted_labels)):
    image = data.permute(1, 2, 0)
    image = image.numpy()
    image = image.reshape(image.shape[0], image.shape[1])

    if label == target_class:
        file_name = f"undesired_{i + 1}.jpg"
        plt.imsave(os.path.join(undesired_folder, file_name), image, cmap="gray")
    else:
        file_name = f"desired_{i + 1}.jpg"
        plt.imsave(os.path.join(desired_folder, file_name), image, cmap="gray")
